In [1]:
import re
import numpy as np
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import string 
import nltk 
nltk.download('vader_lexicon')
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
import warnings 
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
import csv
from nltk.sentiment.vader import SentimentIntensityAnalyzer
warnings.filterwarnings("ignore", category=DeprecationWarning )

%matplotlib inline

[nltk_data] Error loading vader_lexicon: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>


In [2]:
#data cleaning 
def clean_text(text):
    text = str(text)
    #filter to allow only aphabets
    text = re.sub(r'[^a-zA-z\']', ' ', text)
    #remove unicode characters 
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    #remove special characters, numbers, punctuations
   # text = re.sub(r'[^a-zA-Z#\']', ' ', text)
    #removing shortwords
    text = ' '.join([w for w in text.split() if len(w)>3])
    #lower case 
    text = text.lower()
    
    return text

In [3]:
#Extracting words
def extract(x):
    words = []
    #Loop over the words in the content 
    for i in x: 
        word = re.findall(r"\b\w{4,225}\b",i)
        words.append(word)
        
    return words

In [4]:
#relevant words extraction 
def relevant(input_word):
    word_freq = nltk.FreqDist(input_word)
    dict_filter =  lambda word_freq, stopwords: dict((w, word_freq[w]) for w in word_freq if w not in STOPWORDS)
    relevant_w = dict_filter(word_freq, STOPWORDS)
    
    return relevant_w

In [75]:
def sentiment_extraction(filename, appname):
    
    
    
    features = pd.read_csv('sentiments_comparison.csv')
    dataset = pd.read_csv(str(filename))
    #dataset = dataset.drop(columns=['isEdited', 'title', 'date'])
    #dataset = dataset.drop(columns=['isEdited', 'title', 'date', 'developerResponse'])
    dataset = dataset.drop(columns=['Company_Name', 'URL', 'Category_URL', 'Time'])
    
    dataset.dropna()
    
    #dataset['clean_review'] = dataset.review.apply(lambda x: np.vectorize(clean_text)(x))
    #dataset['clean_review'] = dataset.content.apply(lambda x: np.vectorize(clean_text)(x))
    dataset['clean_review'] = dataset.Comments.apply(lambda x: np.vectorize(clean_text)(x))
    
    #positive_w = extract(dataset['clean_review'][dataset['rating']>3])
    #negative_w = extract(dataset['clean_review'][dataset['rating']<3])
    
    #positive_w = extract(dataset['clean_review'][dataset['score']>3])
    #negative_w = extract(dataset['clean_review'][dataset['score']<3])
    
    positive_w = extract(dataset['clean_review'][dataset['Star_rating']>3])
    negative_w = extract(dataset['clean_review'][dataset['Star_rating']<3])
    
    positive_w = sum(positive_w, [])
    negative_w = sum(negative_w, [])
    
    allWords = []
    allWords.append(positive_w)
    allWords.append(negative_w)
    allWords = sum(allWords, [])
    
    relevant_w = relevant(allWords)
    
    relevant_words = pd.DataFrame({'Word_freq': list(relevant_w.keys()),
                              'Word_Count':list(relevant_w.values()),
                             })
    
    sid = SentimentIntensityAnalyzer()

    texts = []
    #neg_word_list = []
    #neu_word_list = [] 
    for word1 in relevant_words['Word_freq']:
        if(sid.polarity_scores(word1)['compound'])>0.5 or (sid.polarity_scores(word1)['compound'])<-0.5:
            texts.append(word1)
            
    
    liking = []
    trust = []
    anger = []
    sadness = []
    fear = []
    
    
    l = [] #liking synonyms
    t = [] #trust synonyms
    a = [] #anger synonyms
    sa = [] #sad synonyms
    f = [] #fear synonyms

    for w in features.liking.values:
        for syn in wordnet.synsets(str(w)):
            for lemma in syn.lemmas():
                l.append(lemma.name())
    for txt in texts: 
        if txt in set(l):
            liking.append(txt)
    liking = list(dict.fromkeys(liking))
    #################################################

    for w1 in features.trust.values:
        for syn_t in wordnet.synsets(str(w1)):
            for lemma1 in syn_t.lemmas():
                t.append(lemma1.name())
    for txt in texts:
        if txt in set(t):
            trust.append(txt)
    trust = list(dict.fromkeys(trust))
    ################################################


    for w4 in features.anger.values:
        for syn_a in wordnet.synsets(str(w4)):
            for lemma4 in syn_a.lemmas():
                a.append(lemma4.name())

    for txt in texts: 
        if txt in set(a):
            anger.append(txt)
    anger = list(dict.fromkeys(anger))
    ######################################################

    for w5 in features.sadness.values:
        for syn_sa in wordnet.synsets(str(w5)):
            for lemma5 in syn_sa.lemmas():
                sa.append(lemma5.name())
    for txt in texts:
        if txt in set(sa):
            sadness.append(txt)
    sadness = list(dict.fromkeys(sadness))

    ########################################################
    for w6 in features.fear.values:
        for syn_f in wordnet.synsets(str(w6)):
            for lemma6 in syn_f.lemmas():
                f.append(lemma6.name())

    for txt in texts:
        if txt in set(f):
            fear.append(txt)
    fear = list(dict.fromkeys(fear))
    
    for duplicate in liking:
        if duplicate in trust:
            trust.remove(duplicate)
        
    for duplicate in trust:
        if duplicate in anger:
            anger.remove(duplicate)

    for duplicate in anger:
        if duplicate in sadness:
            sadness.remove(duplicate)


    for duplicate in sadness:
        if duplicate in fear:
            fear.remove(duplicate)
            
    liking_sentiment = pd.DataFrame({'liking':liking})
    trust_sentiment = pd.DataFrame({'trust':trust})
    anger_sentiment = pd.DataFrame({'anger':anger})
    sadness_sentiment = pd.DataFrame({'sadness':sadness})
    fear_sentiment = pd.DataFrame({'fear':fear})


    sentiments = pd.concat([liking_sentiment, trust_sentiment, anger_sentiment, sadness_sentiment, fear_sentiment], axis = 1)
    
     #save as csv file in specific folder
    sentiments.to_csv(appname, index=None)
    return sentiments

   

In [76]:
sentiment_extraction('../../mHealth_google_app/A39 - e-Anatomy.csv', 'app_sentiments/A39_sentiment.csv')

,liking,trust,anger,sadness,fear
0,best,NaN,idiot,NaN,NaN
1,great,NaN,NaN,NaN,NaN
2,excellent,NaN,NaN,NaN,NaN
3,perfect,NaN,NaN,NaN,NaN
4,love,NaN,NaN,NaN,NaN
5,awesome,NaN,NaN,NaN,NaN
6,free,NaN,NaN,NaN,NaN
7,superb,NaN,NaN,NaN,NaN
8,beautiful,NaN,NaN,NaN,NaN
9,wonderful,NaN,NaN,NaN,NaN
